# AWS boto3 Authentication with Assume Role and MFA Token

In [3]:
import os
import boto3

assume_role_arn = os.environ['AWS_ROLE_ARN']
session_name = os.environ['AWS_ROLE_SESSION_NAME']
mfa_serial_number = os.environ['MFA_SERIAL_NUMBER']

# Get MFA token
mfa_totp = input("Enter the MFA code: ")


def role_arn_to_session(*args):
    """
    The function requires the following arguments:
        role_arn_to_session(
            assume_role_arn - 'arn:aws:iam::<ACCOUNT_NUMBER>:role/example-role',
            session_name - Specifies the name to attach to the role session
            mfa_serial_number - ARN of the MFA device,
            mfa_totp - MFA Time-based one-time password
        )
    """
    
    # Create IAM client
    sts_client = boto3.client('sts')
    response = sts_client.assume_role(
        RoleArn=assume_role_arn,
        RoleSessionName=session_name,
        SerialNumber=mfa_serial_number,
        TokenCode=mfa_totp,
    )
    
    temp_credentials = response["Credentials"]
    print(f"Assumed role {assume_role_arn} and got temporary credentials.")
    return temp_credentials


temp_credentials = role_arn_to_session(assume_role_arn, session_name, mfa_serial_number, mfa_totp)

Enter the MFA code:  004581


Assumed role arn:aws:iam::212762837485:role/allow-ops-admin-access-from-other-accounts and got temporary credentials.


In [4]:
# Create the S3 client
s3_resource = boto3.resource('s3',
                             aws_access_key_id=temp_credentials["AccessKeyId"],
                             aws_secret_access_key=temp_credentials["SecretAccessKey"],
                             aws_session_token=temp_credentials["SessionToken"])

print(f"Listing buckets for the assumed role's account:")
for bucket in s3_resource.buckets.all():
    print(bucket.name)

Listing buckets for the assumed role's account:
caspar-health-dev-auto-created-accounts
caspar-health-dev-bulk-patients
caspar-health-dev-hl7
caspar-health-dev-ktl-reports
caspar-health-dev-openvpn-backup
caspar-health-dev-terraform-state
caspar-health-dev-therapy-modules
caspar-health-dev-therapy-modules-import
caspar-health-dev-therapy-modules-recalculation
caspar-health-dev-videos
caspar-health-dev-videos-abr-eu-central-1
caspar-health-dev-videos-abr-eu-central-1-cloudfront-logs
caspar-health-dev-videos-abr-eu-central-1-mpd-cloudfront-logs
caspar-health-dev-videos-cloudfront-logs
caspar-health-dev-videos-input-eu-central-1
caspar-health-macie-results-dev
datadog-forwarder-forwarderbucket-1mck5zbf9pq1p
datadog-forwarder-media-converter-forwarderbucket-71kprxkph1f
lacework-agentless-scanning-bucket-a17eca26
textract-console-eu-central-1-296c327b-154a-43a5-b192-aa9b164ee


In [5]:
# Create the Textract client

from datetime import datetime

client = boto3.client('textract', region_name='eu-central-1')

response_textract = client.list_adapters(
    AfterCreationTime=datetime(2015, 1, 1),
    BeforeCreationTime=datetime(2015, 1, 1),
    MaxResults=123
)
print(response_textract)

{'Adapters': [], 'ResponseMetadata': {'RequestId': '16f45392-53c8-45df-9555-f7c82d1e6312', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '16f45392-53c8-45df-9555-f7c82d1e6312', 'content-type': 'application/x-amz-json-1.1', 'content-length': '15', 'date': 'Thu, 16 Nov 2023 15:23:05 GMT'}, 'RetryAttempts': 0}}
